# Работа с Docling


*   Сслыка на официальный сайт с документацией https://docling-project.github.io/docling/
*   Сслыка на официальный GitHub https://github.com/docling-project/docling

In [7]:
# Запускаем установку библиотеки docling с помощью менеджера пакетов pip
# Флаг -q (quiet) используется, чтобы скрыть длинный лог установки
!pip install docling -q

In [8]:
# Импортируем основной класс DocumentConverter, который управляет процессом парсинга.
# И объект-обертку PdfFormatOption, внутри которого будут наши настройки
from docling.document_converter import DocumentConverter, PdfFormatOption

# Импортируем класс PdfPipelineOptions для детальной настройки обработки PDF
from docling.datamodel.pipeline_options import PdfPipelineOptions

# Импортируем перечисление InputFormat, чтобы указать, к какому формату применять настройки
from docling.datamodel.base_models import InputFormat

# Создаем экземпляр настроек пайплайна (процесса обработки) для PDF
pipeline_options = PdfPipelineOptions()

# Отключаем OCR (оптическое распознавание текста), так как это самая медленная операция
pipeline_options.do_ocr = False

# Включаем или отключаем распознавание структуры таблиц
pipeline_options.do_table_structure = True

# Инициализируем основной конвертер документов
doc_converter = DocumentConverter(
    # Передаем словарь настроек форматов
    format_options={
        # InputFormat.PDF связываем с оберткой PdfFormatOption, куда кладем наши настройки
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options
        )
    }
)

# Выводим сообщение, что конвертер готов к работе
print("Конвертер успешно инициализирован.")

Конвертер успешно инициализирован.


# Парсинг PDF

In [9]:
# Импортируем модуль files из пакета google.colab, необходимый для загрузки файлов в облако
from google.colab import files

# Выводим сообщение-инструкцию для пользователя
print("Пожалуйста, загрузите PDF-файл с вашего компьютера (нажмите кнопку ниже):")

# Вызываем метод upload, который создает виджет для загрузки файлов
uploaded = files.upload()

# Получаем имя загруженного файла
# Функция upload возвращает словарь, мы берем имя первого ключа (имя файла)
filename = next(iter(uploaded))

# Сообщаем пользователю, что файл успешно сохранен в среде Colab
print(f"Файл '{filename}' успешно загружен и готов к обработке.")

Пожалуйста, загрузите PDF-файл с вашего компьютера (нажмите кнопку ниже):


Saving instrukciya-samsung-galaxy-s23-ultra-6.pdf to instrukciya-samsung-galaxy-s23-ultra-6 (1).pdf
Файл 'instrukciya-samsung-galaxy-s23-ultra-6 (1).pdf' успешно загружен и готов к обработке.


Важный момент. Работа Docling на CPU может быть достаточно долгой (от 5 минут), поэтому рекомендуется включить ускорение в Google Colab. Вверху нажмите на "Среда выполнения" -> "Сменить среду выполнения" -> выберите Графический процессор T4. Либо можно воспользоваться пакетом docling-parse, упрощённой версией для парсинга PDF документов (ячейка с кодом будет далее)

In [ ]:
# Передаем имя локального файла в метод convert
result_local = doc_converter.convert(filename)

# Экспортируем распознанное содержимое документа в формат Markdown
markdown_local = result_local.document.export_to_markdown()

# Выводим визуальный разделитель для удобства чтения вывода
print("-" * 30)

# Выводим заголовок перед демонстрацией текста
print(f"Текст из файла {filename} (первые 500 символов):\n")

# Печатаем первые 500 символов полученного текста для проверки результата
print(markdown_local[:500])

# Применение пакета docling-parse

Ссылка на официальный GitHub https://github.com/docling-project/docling-parse

In [10]:
# Импортируем класс TextCellUnit, который используется для указания того, какие единицы текста (символы, слова, строки) мы хотим обрабатывать
from docling_core.types.doc.page import TextCellUnit

# Импортируем DoclingPdfParser (класс для парсинга PDF) и PdfDocument (класс для представления документа)
# Эти компоненты берутся из пакета docling_parse быстрого, низкоуровневого парсера
from docling_parse.pdf_parser import DoclingPdfParser, PdfDocument

# Создаем экземпляр парсера PDF
parser = DoclingPdfParser()

# Используем парсер для загрузки PDF-документа
pdf_doc: PdfDocument = parser.load(
    path_or_stream=filename
)

# Создаем пустой список, в который будем собирать текст каждой страницы.
all_text = []

# Начинаем итерацию по страницам загруженного документа
# page_no — номер страницы (начиная с 0), pred_page — объект, представляющий страницу
for page_no, pred_page in pdf_doc.iterate_pages():

  # Создаем пустой список для сбора слов текущей страницы
    page_words = []

    # Итерируемся по текстовым ячейкам на текущей странице, выбирая единицы типа WORD
    for word in pred_page.iterate_cells(unit_type=TextCellUnit.WORD):
        # Добавляем текст каждого слова в список слов страницы
        page_words.append(word.text)

    # Объединяем все слова на странице в одну строку, разделяя их пробелами
    page_text = " ".join(page_words)

    # Добавляем полученный текст страницы в общий список
    all_text.append(page_text)

    # Выводим сообщение о том, что страница обработана, для демонстрации прогресса
    print(f"Страница {page_no + 1} обработана.")

# Объединяем текст всех страниц в один финальный документ, разделяя страницы двойным переводом строки
final_document_text = "\n\n".join(all_text)

# Выводим разделитель
print("-" * 50)

# Выводим начало извлеченного текста для демонстрации
print("Извлеченный текст (первые 1000 символов):\n")
print(final_document_text[:1000])

Страница 2 обработана.
Страница 3 обработана.
Страница 4 обработана.
Страница 5 обработана.
Страница 6 обработана.
Страница 7 обработана.
Страница 8 обработана.
Страница 9 обработана.
Страница 10 обработана.
Страница 11 обработана.
Страница 12 обработана.
Страница 13 обработана.
Страница 14 обработана.
Страница 15 обработана.
Страница 16 обработана.
Страница 17 обработана.
Страница 18 обработана.
Страница 19 обработана.
Страница 20 обработана.
Страница 21 обработана.
Страница 22 обработана.
Страница 23 обработана.
Страница 24 обработана.
Страница 25 обработана.
Страница 26 обработана.
Страница 27 обработана.
Страница 28 обработана.
Страница 29 обработана.
Страница 30 обработана.
Страница 31 обработана.
Страница 32 обработана.
Страница 33 обработана.
Страница 34 обработана.
Страница 35 обработана.
Страница 36 обработана.
Страница 37 обработана.
Страница 38 обработана.
Страница 39 обработана.
Страница 40 обработана.
Страница 41 обработана.
Страница 42 обработана.
Страница 43 обработана.


# Парсинг сайта

In [11]:
# Задаем переменную с адресом веб-сайта для парсинга (пример: статья из Википедии про Python)
source_url = "https://ru.wikipedia.org/wiki/Python"

# Выводим сообщение о начале процесса
print(f"Начинаю парсинг сайта: {source_url}")

# Запускаем конвертацию по ссылке. Метод convert автоматически определяет формат (HTML)
# result - это объект, содержащий результат конвертации
result_html = doc_converter.convert(source_url)

# Извлекаем содержимое документа в формате Markdown
markdown_text = result_html.document.export_to_markdown()

# Выводим разделитель для наглядности в консоли
print("-" * 30)

# Выводим первые 500 символов полученного текста для демонстрации
print("Результат (первые 500 символов):\n")
print(markdown_text[:500])

# Выводим разделитель
print("-" * 30)

Начинаю парсинг сайта: https://ru.wikipedia.org/wiki/Python


------------------------------
Результат (первые 500 символов):

# Python

Материал из Википедии - свободной энциклопедии

[Перейти к навигации](#mw-head)

[Перейти к поиску](#searchInput)

У этого термина существуют и другие значения, см. [Python (значения)](/wiki/Python_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F)) .

| Python                                                                                                                                                                                                                                     
------------------------------
